<a href="https://colab.research.google.com/github/Idamfuadi/Capstone-Project-Dilaut/blob/ml/Dilaut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dilaut - Bangkit Capstone Project
A catalog fish prices application which contains current month price and price fishes prediction for next month. The core of our idea is to help fishermen to sell their catch directly to the community, and maintain price stability among the community. Hopefully, it would help by providing transparency of fish prices, so that fishermen are not trapped by the prices given by middlemen, especially in Gresik, Jawa Timur.

In [ ]:
import numpy as np
import pandas as pd 
import time
import math
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from keras.applications.inception_v3 import InceptionV3

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
#load dataset
data = pd.read_csv('Dilaut Dataset.csv')
data

In [ ]:
#filling NaN data
median = data.median()
data.fillna(median, inplace=True)
data

In [ ]:
dm = data.filter(['Mas'])
dmas = dm.values
training_dm_len = math.ceil(len(dmas) * .8)
training_dm_len 

In [ ]:
#scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df)
scaled_data

In [ ]:
#create the training data set
train_data = scaled_data[0:training_dm_len, :]

#split the data into X_train and y_train dataset
X_train = []
y_train = []

for i in range(3, len(train_data)):
  X_train.append(train_data[i-3:i, 0])
  y_train.append(train_data[i, 0])
  if i <= 4:
    print(X_train)
    print(y_train)
    print()


In [ ]:
#convert the X_train and y_train to numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
#reshape the data
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train.shape

In [ ]:
#create the model
model = Sequential()
model.add(LSTM(128, return_sequences= True,activation='relu',input_shape =(X_train.shape[1], 1)))
model.add(LSTM(25, return_sequences= False,activation='relu'))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='Adam')
print(model.summary())

In [ ]:
#train the model
model.fit(X_train, y_train, epochs=200, batch_size=2)

In [ ]:
#create the testing data set
test_data = scaled_data[training_dm_len - 3: , :]

#split the data into X_testing and y_testing dataset
X_test = []
y_test = dmas[training_dm_len:, :]

for i in range(3, len(test_data)):
  X_test.append(test_data[i-3:i, 0])


In [ ]:
y_test

In [ ]:
X_test = np.array(X_test)
X_test

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test

In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

In [ ]:
train = dm[:training_dm_len]
valid = dm[training_dm_len:]
valid['Predictions'] = predictions
plt.xlabel('Date')

In [ ]:
valid